In [ ]:
# !pip install nltk
# !pip install pymorphy2

In [ ]:
import numpy as np
import pandas as pd


import re 
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords


from google.colab import drive
drive.mount('/content/drive')
link = '/content/drive/My Drive/Colab Notebooks/data/bl/'                        # гитхаб

In [ ]:
class Blacklist:
  """
  На вход получает строку, метод bad_words выдаёт словаь с подсчётом слов из ЧС, есть методы для обновления списка ругательств и угроз, есть метод для удаления ругательств
  """

  def __init__(self, text, return_badwords=False):
    
    patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
    with open(link + 'normal_forms_obscene_corpus.txt') as f:                        # гитхаб
      normal_forms_obs = set(f.read().lower().splitlines())
    with open(link + 'normal_forms_threats_corpus.txt') as f:
      normal_forms_threats = set(f.read().lower().splitlines())

    self.text = re.sub(patterns, ' ', text.lower())
    self.blacklist = normal_forms_obs | normal_forms_threats | set(['ты', 'твой'])
    self.bl_threats = normal_forms_threats
    self.return_badwords = return_badwords

  @staticmethod
  def lemmatize(text):
      """
      Приведение слов в нормальню форму
      """
      stopwords_ru = set(stopwords.words("russian")) - set(['ты'])
      morph = MorphAnalyzer()
      tokens = []
      for token in text.split():
        if token and token not in stopwords_ru:
          token = token.strip()
          token = morph.normal_forms(token)[0]
          tokens.append(token)
      return tokens
  

  def bad_words(self):
    """
    Подсчёт слов из blacklist, obs - мат, threats - угрозы, you - обращение на "ты", cho - чо/че/чё
    """
    words = self.lemmatize(self.text)
    bad_words = [w for w in words if w in self.blacklist]
    threats = [w for w in bad_words if w in self.bl_threats]
    you = [w for w in bad_words if w in set(['ты', 'твой'])]
    cho = [w for w in self.text.split() if w in set(['че', 'чё', 'чо'])]

    if self.return_badwords:
      return {'obs': len(bad_words) - len(threats) - len(you), 'threats': len(threats), 'you': len(you), 'cho': len(cho)}, bad_words


    return {'obs': len(bad_words) - len(threats) - len(you), 'threats': len(threats), 'you': len(you), 'cho': len(cho)}

  def update_obscene(self, new_text):
    """
    Добавление новых слов в blacklist, на вход подавать строку слов через пробел
    """
    with open(link + 'normal_forms_obscene_corpus.txt') as f:
      normal_forms_obs = set(f.read().lower().splitlines())
    

    obscene = set(self.lemmatize(new_text)) | set(new_text.split())
    self.blacklist |= obscene

    new_normal_forms_obs = obscene | normal_forms_obs
    with open(link + 'normal_forms_obscene_corpus.txt', 'w') as f:                   # гитхаб
      f.write("\n".join(new_normal_forms_obs))

  def update_threats(self, new_text):
    """
    Добавление новых слов в blacklist, на вход подавать строку слов через пробел
    """
    with open(link + 'normal_forms_threats_corpus.txt') as f:
      normal_forms_threats = set(f.read().lower().splitlines())


    threats = set(self.lemmatize(new_text)) | set(new_text.split())
    self.blacklist |= threats


    new_normal_forms_threats = threats | normal_forms_threats
    with open(link + 'normal_forms_threats_corpus.txt', 'w') as f:                   # гитхаб
      f.write("\n".join(new_normal_forms_threats))
    

  def del_obscene(self, good_words):
      """
      Удаление строки слов из списка ругательств
      """
      with open(link + 'normal_forms_obscene_corpus.txt') as f:
        normal_forms_obs = set(f.read().lower().splitlines())
      good_words = set(good_words.split()) # | set(self.lemmatize(good_words))
      self.blacklist -= good_words

      new_normal_forms_obs = normal_forms_obs - good_words
      with open(link + 'normal_forms_obscene_corpus.txt', 'w') as f:                   # гитхаб
        f.write("\n".join(new_normal_forms_obs))

  def del_obscene(self, good_words):
      """
      Удаление строки слов из списка угроз
      """
      with open(link + 'normal_forms_threats_corpus.txt') as f:
        normal_forms_threats = set(f.read().lower().splitlines())
      good_words = set(good_words.split())  # | set(self.lemmatize(good_words))
      self.blacklist -= good_words

      new_normal_forms_threats = normal_forms_threats - good_words
      with open(link + 'normal_forms_threats_corpus.txt', 'w') as f:                   # гитхаб
        f.write("\n".join(new_normal_forms_threats))

bl = Blacklist(text)
bl.bad_words()